In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax

In [3]:
cie_9 = pl.read_excel(
    "../data/external/Esquema_Registro-2023.xlsx",
    sheet_name="Anexo 2",
    read_csv_options={"skip_rows": 5},
)

cie_10 = pl.read_excel("../data/external/CIE-10 - sin_puntos_y_X.xlsx")

## 1. Análisis Sociodemográfico de Pacientes Atendidos en el Hospital del Tórax

En este cuadernillo se quiere ver el perfil sociodemográfico de los pacientes hospitalizados
en el Hospital del Tórax. Se utilizará la base de datos pública de egresos hospitalarios DEIS
entre 2001 y 2020. Las preguntas que se quieren responder son:

- ¿Cómo es la distribución geográfica de los pacientes atendidos en el INT?
- ¿Cuál es el rango etario más atendido en el INT?
- ¿Cuál es el género más atendido en el INT?
- ¿Cuál es la previsión más atendida en el INT?
- ¿Cuál es la letra de FONASA más atendida en el INT?

Para responder estas preguntas se realizará una agrupación de los datos con todas las variables
sociodemográficas de interés para el análisis.

In [4]:
# Define todas las variables que se quieren saber

VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS = ["ANO_EGRESO", "REGION_RESIDENCIA", "COMUNA_RESIDENCIA"]
VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS = ["SEXO", "EDAD_CATEGORIA", "PREVISION", "BENEFICIARIO"]
VARIABLE_A_CONTAR = "DIAG1"

agrupacion_total = (
    VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS
    + VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS
    + [VARIABLE_A_CONTAR]
)

Luego, se genera la consulta de la base de datos. Para reducir el coste computacional, se filtra
la base de datos únicamente para los datos del INT. Posteriormente, se agrupa la base de datos
con las variables sociodemográficas de interés y se cuenta la ocurrencia de cada grupo.

In [14]:
desglose_sociodemografico = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)
    .group_by(agrupacion_total)
    .agg([pl.count(), pl.col("INTERV_Q").sum().alias("n_int_q")])
    .sort(["ANO_EGRESO"])
    .collect(streaming=True)
)

In [17]:
display(desglose_sociodemografico.sum())

ANO_EGRESO,REGION_RESIDENCIA,COMUNA_RESIDENCIA,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,n_int_q
i64,i64,i64,str,str,str,str,str,u32,i64
148543932,843571,855600363,null,null,null,null,null,96742,24101


Los resultados indican que en todo el periodo analizado el INT presentó 96742 Egresos Hospitalarios
(columna "count") y 24101 Egresos con Procedimientos Quirúrgicos. Esto está en concordancia con el
cuadernillo de egresos hospitalarios.

In [6]:
desglose_sociodemografico_cie = (
    desglose_sociodemografico.join(
        cie_9, how="left", left_on="COMUNA_RESIDENCIA", right_on="Código Comuna"
    )
    .with_columns(pl.col(["PREVISION", "Nombre Comuna"]).str.to_titlecase())
    .drop("Código Región")
)

desglose_sociodemografico_cie = desglose_sociodemografico_cie.join(
    cie_10, how="left", left_on="DIAG1", right_on="Código"
)

desglose_sociodemografico_cie = desglose_sociodemografico_cie.rename(
    {"REGION_RESIDENCIA": "codregion", "COMUNA_RESIDENCIA": "cod_comuna"}
)

In [7]:
display(desglose_sociodemografico_cie.head())

ANO_EGRESO,codregion,cod_comuna,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,Nombre Comuna,Nombre Región,Descripción,Categoría,Sección,Capítulo
i64,i64,i64,str,str,str,str,str,u32,str,str,str,str,str,str
2001,13,13118,"""Mujer""","""(20, 30]""","""Fonasa""","""A""","""J189""",1,"""Macul""","""Metropolitana …","""Neumonía, no e…","""J18 NEUMONÍA, …","""J09-J18 GRIPE…","""Cap.10 ENFERM…"
2001,13,13101,"""Hombre""","""(30, 40]""","""Fonasa""","""A""","""J90X""",1,"""Santiago""","""Metropolitana …","""Derrame pleura…","""J90 DERRAME PL…","""J90-J94 OTRAS…","""Cap.10 ENFERM…"
2001,13,13105,"""Mujer""","""(40, 50]""","""Fonasa""","""A""","""I509""",3,"""El Bosque""","""Metropolitana …","""Insuficiencia …","""I50 INSUFICIEN…","""I30-I52 OTRAS…","""Cap.09 ENFERM…"
2001,13,13123,"""Hombre""","""(70, 80]""","""Desconocido""",null,"""J449""",1,"""Providencia""","""Metropolitana …","""Enfermedad pul…","""J44 OTRAS ENFE…","""J40-J47 ENFER…","""Cap.10 ENFERM…"
2001,13,13123,"""Mujer""","""(50, 60]""","""Fonasa""","""B""","""G473""",2,"""Providencia""","""Metropolitana …","""Apnea del sueñ…","""G47 TRASTORNOS…","""G40-G47 TRAST…","""Cap.06 ENFERM…"


En los resultados se indica el perfil sociodemográfico para cada combinación de variable presente
en la base de datos. A modo de ejemplo, hubo 1 ocurrencia en el 2001 de una paciente de la Región
número 13 y Comuna 13605 que tenía entre 70 a 80 años que tenía PREVISION FONASA Letra B diagnosticada
con el diagnóstico K810.

Uno de los diagnósticos de interés para el Instituto es el de código "C341".

In [8]:
display(desglose_sociodemografico_cie.filter(pl.col("DIAG1") == "C341"))
print(desglose_sociodemografico_cie.filter(pl.col("DIAG1") == "C341").select(pl.col("count").sum()))

ANO_EGRESO,codregion,cod_comuna,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,Nombre Comuna,Nombre Región,Descripción,Categoría,Sección,Capítulo
i64,i64,i64,str,str,str,str,str,u32,str,str,str,str,str,str
2001,8,8101,"""Hombre""","""(60, 70]""","""Desconocido""",null,"""C341""",1,"""Concepción""","""del Biobío""","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,3,3101,"""Hombre""","""(60, 70]""","""Fonasa""","""C""","""C341""",1,"""Copiapo""","""de Atacama""","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,13,13128,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",1,"""Renca""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,13,13101,"""Hombre""","""(50, 60]""","""Fonasa""","""A""","""C341""",1,"""Santiago""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,13,13113,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",1,"""La Reina""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,3,3102,"""Hombre""","""(50, 60]""","""Fonasa""","""A""","""C341""",1,"""Caldera""","""de Atacama""","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2001,13,13106,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",1,"""Estación Centr…","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2003,6,6101,"""Hombre""","""(50, 60]""","""Fonasa""","""A""","""C341""",1,"""Rancagua""","""del Libertador…","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2003,13,13110,"""Mujer""","""(50, 60]""","""Fonasa""","""B""","""C341""",1,"""La Florida""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"


shape: (1, 1)
┌───────┐
│ count │
│ ---   │
│ u32   │
╞═══════╡
│ 1709  │
└───────┘


Por lo tanto, se muestra cada desglose sociodemográfico de los pacientes atendidos. Además,
se indica que la suma de cada conteo de grupo suma 1709 entre 2001 y 2020. Esto está en concordancia
con los egresos totales (1709) para el diagnóstico mencionado en el periodo mencionado.

In [9]:
display(desglose_sociodemografico_cie.null_count())

ANO_EGRESO,codregion,cod_comuna,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,Nombre Comuna,Nombre Región,Descripción,Categoría,Sección,Capítulo
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,3766,0,0,0,0,5,5,5,5


Se muestra que la unión con los diccionarios CIE se realizó correctamente. Solamente 5 registros
carecen de una información CIE sobre el diagnóstico. Por otro lado, todos los registros tienen
información sobre la comuna y región.

In [10]:
desglose_sociodemografico_cie.write_csv(
    "../data/interim/sociodemografico/desglose_sociodemografico.csv",
    separator=";",
)